In [1]:
# from BCBio.GFF import GFFExaminer

https://gffpandas.readthedocs.io/en/latest/tutorial.html

In [1]:
import gffpandas.gffpandas as gffpd
import pandas as pd

In [2]:
annotation = gffpd.read_gff3('YPS1009_S288C_v4_polished.gff')
annot_s288c = gffpd.read_gff3('saccharomyces_cerevisiae_R64-3.gff')

C:\Users\user\anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',


In [3]:
annotation.df['type'].value_counts()

CDS                                7178
gene                               6707
mRNA                               6707
noncoding_exon                      493
tRNA_gene                           321
tRNA                                321
transposable_element_gene            99
transposable_element                 99
snoRNA                               77
snoRNA_gene                          77
plus_1_translational_frameshift      47
five_prime_UTR_intron                24
ncRNA_gene                           23
ncRNA                                23
snRNA_gene                            6
snRNA                                 6
uORF                                  6
telomeric_repeat                      3
intein_encoding_region                1
Name: type, dtype: int64

In [4]:
annot_s288c.df['type'].value_counts()

mRNA                                  11119
CDS                                    7066
gene                                   6607
ARS                                     543
noncoding_exon                          489
long_terminal_repeat                    384
intron                                  378
tRNA                                    299
tRNA_gene                               299
ARS_consensus_sequence                  196
transposable_element_gene                91
transposable_element                     91
region                                   78
snoRNA_gene                              77
snoRNA                                   77
telomere                                 64
LTR_retrotransposon                      50
plus_1_translational_frameshift          47
X_element                                32
centromere                               32
telomeric_repeat                         31
X_element_combinatorial_repeat           28
rRNA                            

## Make df by features YPS1009

In [5]:
TE_df = annotation.filter_feature_of_type(['transposable_element_gene'])
sno_df = annotation.filter_feature_of_type(['snoRNA_gene'])
tRNA_df = annotation.filter_feature_of_type(['tRNA_gene'])
gene_df = annotation.filter_feature_of_type(['gene'])

In [6]:
gene_attr = gene_df.attributes_to_columns()
gene = gene_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [7]:
gene_attr['valid_ORFs'].value_counts()
# so valid_ORFs in gene does count itself and I should look carefully at the 439 genes with no valid ORF

1    6175
0     530
Name: valid_ORFs, dtype: int64

In [8]:
TE_attr = TE_df.attributes_to_columns()
sno_attr = sno_df.attributes_to_columns()
tRNA_attr = tRNA_df.attributes_to_columns()

In [9]:
tRNA_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'sequence_ID'],
      dtype='object')

In [10]:
TE = TE_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID', 'valid_ORFs']]

sno = sno_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

tRNA = tRNA_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

In [11]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
def lat_to_arabic(DF):
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch', 'MANY': 7}
    seq_id = DF['seq_id']
    ch = []
    contig = []
    for i in seq_id:
        chrom, cont = i.split('_')
        contig.append(cont)
        chrom = chrom.replace('chr', '')
        if '-' in chrom:
            chrom, e = chrom.split('-')
        ch.append(Latin_to_arabic[chrom])
    DF['Chromosome'] = ch
    DF['Contig'] = contig
    return DF

In [12]:
gene = lat_to_arabic(gene)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig


In [13]:
TE = lat_to_arabic(TE)
sno = lat_to_arabic(sno)
tRNA = lat_to_arabic(tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig
C:\Users\user\AppData\Local\Temp\ipykernel_12224\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [14]:
gene.head()

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
0,chrII-2_tig00004927,8297,10119,YBR298C,"MAL31,MAL3T,MALT,maltose%20permease",Maltose%20permease%3B%20high-affinity%20maltos...,"GO:0000017,GO:0005352,GO:0071944,SO:0000704",.,-,.,...,Maltose%20permease,1,MAL31,None,Verified,None,0.946,0,2,tig00004927
3,chrII-2_tig00004927,10675,10894,YBR298C-A,None,Putative%20protein%20of%20unknown%20function%3...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative%20protein%20of%20unknown%20function,1,None,None,Uncharacterized,None,0.981,0,2,tig00004927
6,chrII-2_tig00004927,10985,12717,YBR299W,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",Maltase%20%28alpha-D-glucosidase%29%3B%20induc...,"GO:0000025,GO:0004575,GO:0005886,GO:0005987,GO...",.,+,.,...,Maltase%20%28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0,2,tig00004927
9,chrII-2_tig00004927,14242,14700,YGR293C,None,Dubious%20open%20reading%20frame%3B%20unlikely...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0,2,tig00004927
12,chrII-2_tig00004927,14660,15022,YGR294W,"PAU12,seripauperin%20PAU12",Protein%20of%20unknown%20function%3B%20member%...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein%20of%20unknown%20function,1,PAU12,None,Verified,None,0.983,1,2,tig00004927


In [15]:
def clean20(DF):
    note = DF['Note']
    dis = DF['display']
    clean = []
    display = []
    for n in note:
        clean.append(n.replace('%20', ' '))
    for d in dis:
        display.append(str(d).replace('%20', ' '))
    DF['Note'] = clean
    DF['display'] = display
    return DF

In [16]:
gene = clean20(gene)
TE = clean20(TE)
sno = clean20(sno)
tRNA = clean20(tRNA)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


## Df by attribute for S288C

In [17]:
TE_df_s288c = annot_s288c.filter_feature_of_type(['transposable_element_gene'])
sno_df_s288c = annot_s288c.filter_feature_of_type(['snoRNA_gene'])
tRNA_df_s288c = annot_s288c.filter_feature_of_type(['tRNA_gene'])
gene_df_s288c = annot_s288c.filter_feature_of_type(['gene'])

In [18]:
gene_attr_s288c = gene_df_s288c.attributes_to_columns()
TE_attr_s288c = TE_df_s288c.attributes_to_columns()
sno_attr_s288c = sno_df_s288c.attributes_to_columns()
tRNA_attr_s288c = tRNA_df_s288c.attributes_to_columns()

In [19]:
TE_attr_s288c.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term', 'curie',
       'dbxref', 'display'],
      dtype='object')

In [20]:
gene_s288c = gene_attr_s288c[['seq_id', 'start', 'end', 'Name', 'gene',  'Alias',  'Note', 'Ontology_term', 'strand', 'phase', 'curie',
        'orf_classification', 'display']]

TE_s288c = TE_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

sno_s288c = sno_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

tRNA_s288c = tRNA_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

In [21]:
def lat_to_arabic_S288(df):
    # This code just convert the chromosome numbering to integer and remove mitochondrial genes.
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch'}
    chLat = df['seq_id']
    ch = []
    for chrom in chLat:
        chrom = chrom.replace('chr', '')
        ch.append(Latin_to_arabic[chrom])
    df['Chromosome'] = ch
    return df

In [22]:
gene_s288c = lat_to_arabic_S288(gene_s288c)
TE_s288c = lat_to_arabic_S288(TE_s288c)
sno_s288c = lat_to_arabic_S288(sno_s288c)
tRNA_s288c = lat_to_arabic_S288(tRNA_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_12224\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [23]:
gene_s288c = clean20(gene_s288c)
TE_s288c = clean20(TE_s288c)
sno_s288c = clean20(sno_s288c)
tRNA_s288c = clean20(tRNA_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_12224\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


# TE

In [24]:
TE.to_excel('TE_liftoff4_yps1009.xlsx')

In [25]:
TE_s288c['Chromosome'].value_counts()

4     17
7     10
16     9
12     8
13     8
15     8
2      6
10     6
5      4
8      4
14     4
1      2
3      2
6      2
9      1
Name: Chromosome, dtype: int64

In [26]:
TE['Chromosome'].value_counts()

7     54
12    14
4     12
5      8
2      2
6      2
15     2
3      1
1      1
8      1
16     1
10     1
Name: Chromosome, dtype: int64

In [27]:
TE['valid_ORFs'].value_counts()

1    61
0    38
Name: valid_ORFs, dtype: int64

In [28]:
TE_count = TE['Chromosome'].value_counts().to_frame()
TE_count = TE_count.rename(columns = {'Chromosome': 'yps1009 TE'})
TE_count_s = TE_s288c['Chromosome'].value_counts().to_frame()
TE_count_s = TE_count_s.rename(columns = {'Chromosome': 'S288C TE'})

In [29]:
TE_count = TE_count_s.join(TE_count)

In [30]:
TE_count

,S288C TE,yps1009 TE
4,17,12.0
7,10,54.0
16,9,1.0
12,8,14.0
13,8,NaN
15,8,2.0
2,6,2.0
10,6,1.0
5,4,8.0
8,4,1.0


In [31]:
TE7_yps = list(TE[TE['Chromosome'] == 7]['Name'])
TE7_s288 = list(TE_s288c[TE_s288c['Chromosome'] == 7]['Name'])

In [32]:
set(TE7_s288) & set(TE7_yps)

{'YGR027W-A', 'YGR027W-B', 'YGR038C-A', 'YGR038C-B', 'YGR161C-C'}

In [33]:
TE['valid_ORFs'].value_counts()

1    61
0    38
Name: valid_ORFs, dtype: int64

In [34]:
TE[TE['Chromosome'] == 1]

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,copy_num_ID,coverage,curie,display,extra_copy_number,sequence_ID,valid_ORFs,Chromosome,Contig
5939,chrI_tig00000033,177837,179152,YDR034C-C,gag%20protein,Retrotransposon TYA Gag gene co-transcribed wi...,"GO:0000943,GO:0003723,GO:0032197,SO:0000704",.,+,.,YDR034C-C_4,0.999,SGD:S000007344,Retrotransposon TYA Gag gene co-transcribed wi...,4,0.973,0,1,tig00000033


# snoRNA

In [35]:
sno_count = sno['Chromosome'].value_counts().to_frame()
sno_count = sno_count.rename(columns = {'Chromosome': 'yps1009 snoRNA'})
sno_count_s = sno_s288c['Chromosome'].value_counts().to_frame()
sno_count_s = sno_count_s.rename(columns = {'Chromosome': 'S288C snoRNA'})
sno_count = sno_count_s.join(sno_count)

In [36]:
sno_count

,S288C snoRNA,yps1009 snoRNA
13,13,13
15,11,11
12,7,7
7,6,6
16,6,6
5,5,5
10,5,5
11,5,5
14,5,5
3,4,4


# tRNA

In [37]:
tRNA['extra_copy_number'].value_counts()

0    299
1     22
Name: extra_copy_number, dtype: int64

In [38]:
tRNA_count = tRNA['Chromosome'].value_counts().to_frame()
tRNA_count = tRNA_count.rename(columns = {'Chromosome': 'yps1009 tRNARNA'})
tRNA_count_s = tRNA_s288c['Chromosome'].value_counts().to_frame()
tRNA_count_s = tRNA_count_s.rename(columns = {'Chromosome': 'S288C tRNARNA'})
tRNA_count = tRNA_count_s.join(tRNA_count)

In [39]:
tRNA_count

,S288C tRNARNA,yps1009 tRNARNA
7,36,38
4,28,30
mitoch,24,26
10,24,27
12,21,22
13,21,22
5,20,21
15,20,20
16,17,18
11,16,18


# GENE

In [40]:
gene['orf_classification'].value_counts()

Verified           5249
Uncharacterized     737
Dubious             719
Name: orf_classification, dtype: int64

In [41]:
dubious = gene[gene['orf_classification'] == 'Dubious']
dubious['valid_ORFs'].value_counts()

1    524
0    195
Name: valid_ORFs, dtype: int64

In [42]:
uncharacterized = gene[gene['orf_classification'] == 'Uncharacterized']
uncharacterized['valid_ORFs'].value_counts()

1    564
0    173
Name: valid_ORFs, dtype: int64

# Verified ORF

In [43]:
verified = gene[gene['orf_classification'] == 'Verified']

In [44]:
verified['valid_ORFs'].value_counts()

1    5087
0     162
Name: valid_ORFs, dtype: int64

In [45]:
verified_s288c = gene_s288c[gene_s288c['orf_classification'] == 'Verified']

In [46]:
verified_s288c.shape

(5216, 14)

In [47]:
verified_valid = verified[verified['valid_ORFs'] == '1']
verified_unvalid = verified[verified['valid_ORFs'] == '0']

verified_valid_count = verified_valid['Chromosome'].value_counts().to_frame()
verified_valid_count = verified_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
verified_unvalid_count = verified_unvalid['Chromosome'].value_counts().to_frame()
verified_unvalid_count = verified_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [49]:
unval_dic = {}
for ind, row in verified_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]    

In [67]:
verified_sum = verified_valid_count.join(verified_unvalid_count)

In [68]:
tmp = []
for ind in verified_sum.index:
    tmp.append(unval_dic[ind])

In [69]:
verified_sum['unvalid genes'] = tmp

In [70]:
verified.head(2)

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
0,chrII-2_tig00004927,8297,10119,YBR298C,"MAL31,MAL3T,MALT,maltose%20permease",Maltose permease%3B high-affinity maltose tran...,"GO:0000017,GO:0005352,GO:0071944,SO:0000704",.,-,.,...,Maltose permease,1,MAL31,None,Verified,None,0.946,0,2,tig00004927
6,chrII-2_tig00004927,10985,12717,YBR299W,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",Maltase %28alpha-D-glucosidase%29%3B inducible...,"GO:0000025,GO:0004575,GO:0005886,GO:0005987,GO...",.,+,.,...,Maltase %28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0,2,tig00004927


In [72]:
missed = []
moved = []
count_miss = []
count_move = []
for Ch in verified_sum.index:
    s288 = verified_s288c[verified_s288c['Chromosome'] == Ch]
    yps = verified[verified['Chromosome'] == Ch]
    tmp = list(set(s288['Name']) - set(yps['Name']))
    miss = []
    mov = []
    others = verified[verified['Chromosome'] != Ch]
    for g in tmp:
        search = others[others['Name'] == g]
        if search.shape[0] >= 1:
            mov.append((g, list(search['Chromosome']), list(search['valid_ORFs'])))
        else:
            miss.append(g)  
    moved.append(mov)
    missed.append(miss)
    count_miss.append(len(miss))
    count_move.append(len(mov))

In [73]:
verified_sum['Number of missing genes'] = count_miss
verified_sum['Missing genes'] = missed
verified_sum['Number of moved genes'] = count_move
verified_sum['Moved genes, new chromosome(s), orf validity'] = moved

In [74]:
verified_sum

,Number valid genes,Number unvalid genes,unvalid genes,Number of missing genes,Missing genes,Number of moved genes,"Moved genes, new chromosome(s), orf validity"
4,673,9,"[YDL247W, YNL331C, YDL227C, YDL080C, YDL039C, ...",0,[],5,"[(YDR385W, [15], [1]), (YDL248W, [13], [0]), (..."
15,467,14,"[YOR388C, YOR249C, YOR244W, YOR245C, YCR106W, ...",3,"[YOR246C, YOR255W, YOR247W]",3,"[(YOR394W, [16], [0]), (YOR390W, [16], [1]), (..."
7,457,5,"[YGL243W, YGL131C, YAR027W, YGR140W, YBR298C]",4,"[YGL133W, YGL137W, YGL134W, YGL136C]",6,"[(YGL263W, [12, 12], [0, 1]), (YGR296W, [5], [..."
12,432,13,"[YGL263W, YLL063C, YLL057C, YLL052C, YLR119W, ...",5,"[YLR160C, YLR157C, YLR158C, YLR154W-C, YLR155C]",3,"[(YLR467W, [5], [1]), (YLR466W, [5], [0]), (YL..."
16,405,8,"[YOR394W, YPL282C, YMR323W, YPL274W, YPL255W, ...",0,[],5,"[(YPL281C, [15], [1]), (YPL279C, [15], [1]), (..."
13,394,8,"[YDL248W, YJR161C, YML118W, YMR019W, YMR104C, ...",0,[],5,"[(YMR105C, [11], [1]), (YMR325W, [15, 16], [1,..."
2,360,7,"[YBR298C, YBR299W, YGR295C, YBR031W, YBR073W, ...",0,[],3,"[(YBL111C, [5], [0]), (YBL108C-A, [9, 12, 12],..."
14,326,18,"[YNL322C, YNL318C, YNL317W, YNL316C, YNL315C, ...",7,"[YNL329C, YNL328C, YNL323W, YNL325C, YNL330C, ...",8,"[(YNL334C, [10], [1]), (YNL333W, [10], [1]), (..."
10,316,12,"[YJL222W, YJL219W, YJL222W, YJL217W, YJL077C, ...",0,[],5,"[(YJR159W, [4], [1]), (YJR161C, [13], [0]), (Y..."
11,291,12,"[YKR102W, YKR103W, YKR104W, YKR105C, YKL161C, ...",0,[],2,"[(YKL224C, [9], [1]), (YKR106W, [3], [1])]"


In [75]:
verified_sum.sum()

Number valid genes                                                                           5087
Number unvalid genes                                                                          162
unvalid genes                                   [YDL247W, YNL331C, YDL227C, YDL080C, YDL039C, ...
Number of missing genes                                                                        21
Missing genes                                   [YOR246C, YOR255W, YOR247W, YGL133W, YGL137W, ...
Number of moved genes                                                                          60
Moved genes, new chromosome(s), orf validity    [(YDR385W, [15], [1]), (YDL248W, [13], [0]), (...
dtype: object

In [76]:
verified_sum.to_excel('verified_genes_summary.xlsx')

# Missing genes dataset

In [85]:
flat_list = [item for sublist in missed for item in sublist]

In [89]:
missing_genes = verified_s288c[verified_s288c['Name'].isin(flat_list)]

In [90]:
missing_genes.to_excel('missing_verified_genes.xlsx')

# Blast

In [92]:
# Import Biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO

In [97]:
ids = []
seq = []
for seq_record in SeqIO.parse("fasta_file/orf_coding_all_R64-3-1_20210421.fasta", "fasta"):
    ids.append(str(seq_record.id))
    seq.append(str(seq_record.seq))

In [104]:
for g in flat_list:
    name = g + '.txt'
    with open(name , "w") as file:
        file.write('> ' + g + '\n')
        file.write(seq[ids.index(g)])
    outfile = 'Blast_results/' + g + '.tab'
    cmd_blastn = NcbiblastnCommandline(query= name, out= outfile, evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
    stdout, stderr = cmd_blastn()
    print("STDOUT: %s" % stdout)
    print("STDERR: %s" % stderr)

STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 
STDOUT: 
STDERR: 


In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR038C_ENA5_genomic.fsa', out='Blast_results/YDR038C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')

In [ ]:
cmd_blastn = NcbiblastnCommandline(query= 'fasta_file/S288C_YDR039C_ENA2_genomic.fsa', out='Blast_results/YDR039C.tab', evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
stdout, stderr = cmd_blastn()
# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

# Duplicated genes



In [79]:
# Many duplicated genes in this version
gene['extra_copy_number'].value_counts()

0    6563
1      94
2      22
3      12
4       7
5       5
6       2
Name: extra_copy_number, dtype: int64

In [80]:
gene[gene['extra_copy_number'] > '1']

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
9,chrII-2_tig00004927,14242,14700,YGR293C,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.989,0,2,tig00004927
21,chrII-2_tig00004927,17994,18290,YER188C-A,None,Putative protein of unknown function,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative protein of unknown function,3,None,None,Uncharacterized,None,0.943,1,2,tig00004927
24,chrIII-2_tig00004935,767,1021,YNR077C,None,Protein of unknown function%2C abundance chang...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein of unknown function%2C abundance chang...,3,None,None,Uncharacterized,None,0.918,1,3,tig00004935
27,chrIII-2_tig00004935,1323,2063,YCL076W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.981,0,3,tig00004935
30,chrIII-2_tig00004935,6426,8271,YCL073C,"GEX1,glutathione%20exchanger",Proton:glutathione antiporter%3B localized to ...,"GO:0005774,GO:0005886,GO:0015299,GO:0034775,GO...",.,-,.,...,Proton:glutathione antiporter,2,GEX1,None,Verified,None,0.990,0,3,tig00004935
33,chrIII_tig00004936,760,1014,YNR077C,None,Protein of unknown function%2C abundance chang...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein of unknown function%2C abundance chang...,2,None,None,Uncharacterized,None,0.918,1,3,tig00004936
36,chrIII_tig00004936,1316,2055,YCL076W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,3,None,None,Dubious,None,0.979,0,3,tig00004936
651,chrIII_tig00004936,310848,311102,YNL337W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,4,None,None,Dubious,None,0.949,1,3,tig00004936
2097,chrII_tig00000010,789332,789467,YHL048C-A,None,Putative protein of unknown function%3B identi...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Putative protein of unknown function,6,None,None,Uncharacterized,None,0.890,0,2,tig00000010
2106,chrIV_tig00000001,1219,1356,YHL048C-A,None,Putative protein of unknown function%3B identi...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative protein of unknown function,4,None,None,Uncharacterized,None,0.920,0,4,tig00000001


In [82]:
dupl = gene[gene.duplicated(subset=['Name'], keep=False) == True]

In [83]:
dupl = dupl.sort_values(by = 'Name')

In [91]:
dupl.to_excel('duplicated_verified_genes.xlsx')

Duplicated genes:

ATP8/Q0080: Subunit 8 of the F0 sector of mitochondrial F1F0 ATP synthase; encoded on the mitochondrial genome; ATP8 and ATP6 mRNAs are not translated in the absence of the F1 sector of ATPase

OLI1/Q0130: F0-ATP synthase subunit c (ATPase-associated proteolipid); encoded on the mitochondrial genome; mutation confers oligomycin resistance; expression is specifically dependent on the nuclear genes AEP1 and AEP2 1 2 3 4
Name Description
OLIgomycin resistance 1

Both on mitochondria so not that relevant to my project.

PAU9/YBL108C-A: Protein of unknown function; member of the seripauperin multigene family encoded mainly in subtelomeric regions; SWAT-GFP and mCherry fusion proteins localize to the endoplasmic reticulum and vacuole respectively

YHR213W-A: Putative protein of unknown function; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching 